In [1]:
import torch
import glob
import os
import numpy as np
import matplotlib.pyplot as plt




In [ ]:
dir_data = "../log_files/BAT-clean_adv_distribution-Madry_loss/BAT-clean_adv_distribution-Madry_loss-Madry_loss--epochs_110-weight_decay_0.0005-lr_max_0.1-epsilon_0.06274509803921569-num_steps_10-step_size_0.00784313725490196-seed_0-/20220410131028_y63mvmvg"

all_data = [torch.load(os.path.join(dir_data, f"dis_measure_{i}.pth")) for i in range(1, 111)]



In [ ]:
print(all_data)

In [ ]:
avg_all_data = np.average([i['bn1.bn_list.0.running_mean'] for i in all_data])

In [ ]:
avg_all_data

In [ ]:
import torch
dir_data = "../log_files/BAT-clean_adv_distribution-Madry_loss/BAT-clean_adv_distribution-Madry_loss-Madry_loss--epochs_110-weight_decay_0.0005-lr_max_0.1-epsilon_0.06274509803921569-num_steps_10-step_size_0.00784313725490196-seed_0-/20220410131028_y63mvmvg/dis_measure_110.pth"

data_ep110 = torch.load(dir_data)

mean_all_layer = [value for key, value in data_ep110.items() if "mean" in key]
var_all_layer = [value for key, value in data_ep110.items() if "var" in key]


import matplotlib.pyplot as plt
plt.figure()
plt.plot(mean_all_layer, label="mean", marker=".")
# plt.title("mean vs layer")
# plt.figure()
plt.plot(var_all_layer, label="var", marker="^")
# plt.title("var vs layer")
plt.ylabel("Wasserstein distance")
plt.xlabel("Depth of layer")
plt.xticks([i for i in range(len(mean_all_layer))])

plt.grid(linestyle='--')
plt.legend()

plt.tight_layout()
plt.show()
plt.savefig("depth_of_layer.png")

In [ ]:

#all_keys = list(all_data[0].keys())
import torch
from scipy.stats import wasserstein_distance as wd
import matplotlib.pyplot as plt

model_path_epoch110='../log_files/BAT-Imagenette-Hybrid_dual_bn/BAT-Imagenette-Hybrid_dual_bn-Madry_mixture_bn--epochs_110-weight_decay_0.0005-lr_max_0.1-epsilon_0.06274509803921569-num_steps_10-step_size_0.00784313725490196-seed_0-/20220410144724_1wuycuyq/modle-epoch110.pt'
model_ep110 = torch.load(model_path_epoch110)

#print(model_ep110.state_dict().keys())


#bn1.bn_list.0.weight   bn1.bn_list.0.bias  bn1.bn_list.0.running_mean   bn1.bn_list.0.running_var
#bn1.bn_list.1.weight   bn1.bn_list.1.bias  bn1.bn_list.1.running_mean   bn1.bn_list.1.running_var
#bn2.bn_list.0.weight   bn2.bn_list.0.bias  bn2.bn_list.0.running_mean   bn2.bn_list.0.running_var
#bn2.bn_list.1.weight   bn2.bn_list.1.bias  bn2.bn_list.1.running_mean   bn2.bn_list.1.running_var
# mean_all_bn_layer = [value for key, value in model_ep110.items() if "bn1.bn_list.0" in key]
# var_all_bn_layer = [value for key, value in model_ep110.items() if "bn1.bn_list.1" in key]

results={}
for k, v in model_ep110.items():
    if "bn_list.0" in k and "num_batches_tracked" not in k:
        bn = v
        bn_corr = model_ep110[k.replace("bn_list.0", "bn_list.1")]
        
        # print(bn.size())
        # print(k)
        # import ipdb; ipdb.set_trace()

        wd_dis = wd(bn.cpu().numpy(), bn_corr.cpu().numpy())
        # print(wd_dis)

        results[k] = wd_dis

#print(results)
    
#import ipdb; ipdb.set_trace()

plt.figure()
plt.plot([v for k, v in results.items() if 'weight' in k], label="weight")
plt.plot([v for k, v in results.items() if 'bias' in k], label="bias")
plt.plot([v for k, v in results.items() if 'running_mean' in k], label="running_mean" )
plt.plot([v for k, v in results.items() if 'running_var' in k], label="running_var")
# print(all_keys[30])
plt.ylabel("Wasserstein distance")
plt.xlabel("Depth of layer")
plt.xticks([i for i in range(len([v for k, v in results.items() if 'weight' in k]))])

plt.grid(linestyle='--')
plt.legend()

plt.tight_layout()
plt.savefig("depth_of_layer.png")


# # plt.twinx()
# plt.plot([i[all_keys[31]] for i in all_data], label="var", color="b")
# print(all_keys[31])


# plt.ylabel("Wasserstein distance")
# plt.xlabel("Epoch")

# plt.grid(linestyle='--')
# plt.legend()

# plt.tight_layout()


In [ ]:
import sys
import torch.nn as nn
import torchvision
import copy

from torchvision import transforms

from resnet_multi_bn_default_pgd import multi_bn_resnet18


# def get_no_current_minibatch_stat_bn(channel):
#     return nn.BatchNorm2d(channel, momentum=1)

# norm_layer = get_no_current_minibatch_stat_bn

# model_for_test = multi_bn_resnet18(norm_layer=norm_layer, bn_names=["pgd", "normal"]).cuda()
# model_path_epoch110='../log_files/BAT-Imagenette-Hybrid_dual_bn/BAT-Imagenette-Hybrid_dual_bn-Madry_mixture_bn--epochs_110-weight_decay_0.0005-lr_max_0.1-epsilon_0.06274509803921569-num_steps_10-step_size_0.00784313725490196-seed_0-/20220410144724_1wuycuyq/modle-epoch110.pt'

# model_state_dict = torch.load(model_path_epoch110)
# model_for_test.load_state_dict(model_state_dict)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
])
trainset = torchvision.datasets.CIFAR10(root='/dev/shm', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True,)
testset = torchvision.datasets.CIFAR10(root='/dev/shm', train=False, download=True, transform=transform_test)
testset.data = testset.data[:1000]
testset.targets = testset.targets[:1000]

test_loader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, )

epsilon = 16/255.


def generate_adv(model,
                x_natural,
                y,
                optimizer=None,
                step_size=0.003,
                epsilon=0.031,
                perturb_steps=10,
                bn_name="pgd",
                ):

    model.eval()

    loss = nn.CrossEntropyLoss()
    adv_images = x_natural.clone().detach()

    adv_images = adv_images + torch.empty_like(adv_images).uniform_(-epsilon, epsilon)
    adv_images = torch.clamp(adv_images, min=0, max=1).detach()

    for _ in range(perturb_steps):
        adv_images.requires_grad = True

        if bn_name:
            outputs = model(adv_images, bn_name)
        else:
            outputs = model(adv_images)

        # Calculate loss
        cost = loss(outputs, y)

        grad = torch.autograd.grad(cost, adv_images,
                                    retain_graph=False, create_graph=False)[0]

        adv_images = adv_images.detach() + step_size*grad.sign()
        delta = torch.clamp(adv_images - x_natural, min=-epsilon, max=epsilon)
        adv_images = torch.clamp(x_natural + delta, min=0, max=1).detach()

    return adv_images




# measure the mismatch between clean and adv
def get_no_current_minibatch_stat_bn(channel):
    return nn.BatchNorm2d(channel, momentum=1)
norm_layer = get_no_current_minibatch_stat_bn
bn_names = ["pgd", "normal"]

model_for_measure = multi_bn_resnet18(norm_layer=norm_layer, bn_names=bn_names, num_classes=10).cuda()
model_path_epoch110='../log_files/BAT-Imagenette-Hybrid_dual_bn/BAT-Imagenette-Hybrid_dual_bn-Madry_mixture_bn--epochs_110-weight_decay_0.0005-lr_max_0.1-epsilon_0.06274509803921569-num_steps_10-step_size_0.00784313725490196-seed_0-/20220410144724_1wuycuyq/modle-epoch110.pt'

model_state_dict = torch.load(model_path_epoch110)

model_for_measure.load_state_dict(model_state_dict)
# get the stat for clean image
print("extra feature")
model_for_measure.train()
for batch_id, (data, target) in enumerate(test_loader):
    data, target = data.cuda(), target.cuda()
    data_adv = generate_adv(model_for_measure, data, target, None, step_size=epsilon/4, epsilon=epsilon, perturb_steps=10)
    model_for_measure.train()
    model_for_measure(data_adv)
    adv_stat = copy.deepcopy(model_for_measure.state_dict())
    model_for_measure(data)
    clean_stat = copy.deepcopy(model_for_measure.state_dict())
    break

from scipy.stats import wasserstein_distance as wass_dis

dis_measure_saver_mean = {}
dis_measure_saver_var = {}

for (adv_bn_stat_key, adv_bn_stat_value), (clean_bn_stat_key, clean_bn_stat_value) in zip(adv_stat.items(), clean_stat.items()):
    if "bn_list.0.running_mean" in adv_bn_stat_key:
        # import ipdb;ipdb.set_trace()
        adv_clean_wass_dis = wass_dis(adv_bn_stat_value.cpu().numpy(), clean_bn_stat_value.cpu().numpy())

        dis_measure_saver_mean[adv_bn_stat_key] = adv_clean_wass_dis
    if "bn_list.0.running_var" in adv_bn_stat_key:
        adv_clean_wass_dis = wass_dis(adv_bn_stat_value.cpu().numpy(), clean_bn_stat_value.cpu().numpy())
        dis_measure_saver_var[adv_bn_stat_key] = adv_clean_wass_dis




plt.figure()
plt.show()





In [ ]:
### plot the trend of affine parameters: layer4.0.bn1.bn_list.0
weight_distantce = []
bias_distance = []
for epoch in range(111):
    ckpt_path = 


In [ ]:

all_keys = list(all_data[0].keys())


# fig, ax1 = plt.subplots()
# ax1.set_ylabel("Wasserstein distance (mean)")
# ax1.plot([i[all_keys[30]] for i in all_data], label="mean", color="r")
# # plt.legend()
# ax2 = ax1.twinx()
# ax2.plot([i[all_keys[31]] for i in all_data], label="var", color="b")
# ax2.set_ylabel("Wasserstein distance (var)")

plt.figure()
plt.plot([i[all_keys[30]] for i in all_data], label="$\mu$", color="r")
print(all_keys[30])

plt.plot([i[all_keys[31]] for i in all_data], label="$\sigma$", color="b")
print(all_keys[31])

plt.plot

plt.ylabel("Wasserstein distance", fontsize=15)
plt.yticks(color="r")
plt.xlabel("Epoch", fontsize=15)
plt.xticks( fontsize=12)
plt.yticks( fontsize=12)

# plt.twinx()
# plt.plot([i[all_keys[31]] for i in all_data], label="var", color="b")
# print(all_keys[31])
# plt.yticks( fontsize=12)


# plt.ylabel("Wasserstein distance (var $\mu$)", color="b", fontsize=15)
# plt.yticks(color="b")
# plt.xlabel("Epoch")

plt.grid(linestyle='--')
# plt.legend()

plt.tight_layout()
plt.savefig("zcs_layer4_firstbn_meanvar.png")

In [ ]:

all_keys = list(all_data[0].keys())


# fig, ax1 = plt.subplots()
# ax1.set_ylabel("Wasserstein distance (mean)")
# ax1.plot([i[all_keys[30]] for i in all_data], label="mean", color="r")
# # plt.legend()
# ax2 = ax1.twinx()
# ax2.plot([i[all_keys[31]] for i in all_data], label="var", color="b")
# ax2.set_ylabel("Wasserstein distance (var)")

plt.figure()
plt.plot([i[all_keys[30]] for i in all_data], label="mean", color="r")
print(all_keys[30])

plt.ylabel("Wasserstein distance (mean $\mu$)", color="r", fontsize=15)
plt.yticks(color="r")
plt.xlabel("Epoch", fontsize=15)
plt.xticks( fontsize=12)
plt.yticks( fontsize=12)

plt.twinx()
plt.plot([i[all_keys[31]] for i in all_data], label="var", color="b")
print(all_keys[31])
plt.yticks( fontsize=12)


plt.ylabel("Wasserstein distance (var $\mu$)", color="b", fontsize=15)
plt.yticks(color="b")
plt.xlabel("Epoch")

plt.grid(linestyle='--')
# plt.legend()

plt.tight_layout()
plt.savefig("zcs_layer4_firstbn_meanvar.png")

In [ ]:

# all_keys = list(all_data[0].keys())

# for layer, key in enumerate(all_keys):

#     plt.figure()
#     plt.plot([i[key] for i in all_data], label="1st BN")
#     plt.title(key + str(layer))
